In [ ]:
%run "Setup_DesignStart.ipynb"

In [ ]:
print(target.get_target_name())
print(target.get_fw_buildtime())
print(target.get_fpga_buildtime())

In [ ]:
# need stream mode and lower clock than normal:
scope.adc.stream_mode = True
scope.adc.samples = 1200000
target.pll.pll_outfreq_set(10E6, 1) # hmm this causes internal MMCM to lose lock

In [ ]:
# if target has MMCM, it won't lock on the PLL1 10 MHz, but it will lock on the CW-PRO-generated 10 MHz clock:
#scope.clock.clkgen_freq = 10e6
#scope.clock.adc_src = "clkgen_x1"

In [ ]:
#target.ss.baud = 38400 # 20 MHz clock
target.ss.baud = 19200 # 10 MHz clock

In [ ]:
# 10M cycles, so for visual plotting purposes it's better to decimate
scope.adc.decimate = 10

In [ ]:
# check ADC lock
scope.clock.reset_adc()
time.sleep(0.1)
assert (scope.clock.adc_locked), "ADC failed to lock"

In [ ]:
target.fpga_write(target.REG_SOFT_TRIG_ENABLE, [1])
target.fpga_write(target.REG_COUNTER_QUICK_START, [1])
target.fpga_write(target.REG_CAPTURE_RAW, [0])
target.fpga_write(target.REG_COUNT_WRITES, [0])
target.fpga_write(target.REG_SOFT_TRIG_PASSTHRU, [1])

In [ ]:
# match on any PC trace packet:
target.set_pattern_match(0, [5, 8, 32, 0, 0, 0, 0, 0], [255, 255, 255, 0, 0, 0, 0, 0])
#target.set_pattern_match(0, [5, 8, 32, 229, 2, 0, 0, 132], [255, 255, 255, 255, 255, 255, 255, 0])

# enable matching rule:
target.fpga_write(target.REG_PATTERN_ENABLE, [1])

target.fpga_write(target.REG_CAPTURE_LEN, int.to_bytes(10200000, length=4, byteorder='little'))

In [ ]:
int.to_bytes(10200000, length=4, byteorder='little')

In [ ]:
target.set_reg('DWT_COMP0', '00000644') # ECC
target.set_reg('DWT_COMP1', '00003ece') # AES

In [ ]:
#target.set_reg('ETM_TEEVR', '000150a0') # COMP0 or COMP1
target.set_reg('ETM_TEEVR', '00000020') # COMP0 only
#target.set_reg('ETM_TEEVR', '00000021') # COMP1 only

In [ ]:
target.arm_trace()

In [ ]:
target.synced()

In [ ]:
target.simpleserial_write('p', bytearray(16), printresult=True)

In [ ]:
# first try AES encryption:
trace = cw.capture_trace(scope, target.ss, plaintext=bytearray(16), key=None)

In [ ]:
from chipwhisperer.common.utils import util

In [ ]:
# now run pmul:
trace = target.capture_ecc_trace(scope, k=util.hexStrToByteArray('ff00000000000000000000000000000000000000000000000000000000000df'), verbose=True)

In [ ]:
# glance at pmul result:
target.simpleserial_write('r', bytearray(32), printresult=True)

In [ ]:
target.fpga_read(target.REG_TRACE_COUNT, 8)

In [ ]:
target.fifo_empty()

In [ ]:
raw = target.read_capture_data()

In [ ]:
len(raw)

In [ ]:
target.print_raw_data(raw)

In [ ]:
times = target.get_rule_match_times(raw, rawtimes=False, verbose=True)

In [ ]:
len(times)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
from bokeh.models import Span

output_notebook(INLINE)
p = figure(plot_width=1500)

size = len(trace.wave)
#size = 1000000
xrange = range(size)
p.line(xrange, trace.wave[:size], line_color="red")

In [ ]:
if scope.clock.adc_src == 'clkgen_x4' or scope.clock.adc_src == 'extclk_x4':
    multiplier = 4/scope.adc.decimate
else:
    multiplier = 1/scope.adc.decimate

In [ ]:
vlines = []
for t in times:
    vlines.append(Span(location=t[0]*multiplier, dimension='height', line_color='black', line_width=2))
p.renderers.extend(vlines)

In [ ]:
show(p)

In [ ]:
prev = times[0][0]
deltas = []
for t in times[1:]:
    deltas.append(t[0] - prev)
    prev = t[0]

In [ ]:
min(deltas), max(deltas)

In [ ]:
max(deltas) - min(deltas)

In [ ]:
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from bokeh.resources import INLINE
from bokeh.models import Span

output_notebook(INLINE)
q = figure(plot_width=1500)

size = len(deltas)
xrange = range(size)
q.line(xrange, deltas, line_color="red")

In [ ]:
show(q)